<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [7]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


for index, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    latitude = row['Lat']
    longitude = row['Long']

    circle = folium.Circle([latitude,longitude], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_site))

    marker = folium.map.Marker(
        [latitude,longitude],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(150,36),  # Adjusted size
            icon_anchor=(0, 0),
            html=f'''
                <div style="
                    font-size: 16px;
                    color: white;
                    background-color: #d35400;
                    padding: 5px;
                    border-radius: 5px;
                    box-shadow: 3px 3px 5px rgba(0,0,0,0.3);
                    ">
                    <b>{launch_site}</b>
                </div>
            '''
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [9]:
import folium
from folium.features import DivIcon

# --- Helper: quick great-circle approximation to equator distance (km)
# Latitude degrees to km is ~111.32 km/deg (good enough for equator distance)
def km_to_equator(lat_deg: float) -> float:
    return abs(float(lat_deg)) * 111.32

# Base map
site_map_p = folium.Map(location=nasa_coordinate, zoom_start=5)

# Feature groups
equator_fg = folium.FeatureGroup(name="Equator")
sites_fg = folium.FeatureGroup(name="Launch Sites")

# -- Equator line (split to avoid anti-meridian artifact)
for seg in [[(0, -180), (0, 0)], [(0, 0), (0, 180)]]:
    folium.PolyLine(
        locations=seg,
        color="blue",
        weight=2,
        opacity=0.8
    ).add_to(equator_fg)

equator_fg.add_to(site_map_p)

# Add sites
bounds = []
for _, row in launch_sites_df.iterrows():
    launch_site = row["Launch Site"]
    lat = float(row["Lat"])
    lon = float(row["Long"])
    bounds.append([lat, lon])

    # Popup + tooltip
    dist_km = km_to_equator(lat)
    popup = folium.Popup(
        f"<b>{launch_site}</b><br>"
        f"Lat: {lat:.5f}, Lon: {lon:.5f}<br>"
        f"Distance to Equator: {dist_km:.1f} km",
        max_width=280
    )
    tooltip = f"{launch_site} (Lat {lat:.5f}, Lon {lon:.5f})"

    # Area highlight (meters)
    folium.Circle(
        location=[lat, lon],
        radius=1000,          # 1 km highlight
        color="#d35400",
        weight=2,
        fill=True,
        fill_opacity=0.15
    ).add_child(popup).add_to(sites_fg)

    # Crisp text label
    folium.Marker(
        location=[lat, lon],
        tooltip=tooltip,
        icon=DivIcon(
            icon_size=(150,36),  # Adjusted size
            icon_anchor=(0, 0),
            html=f'''
                <div style="
                    font-size: 16px;
                    color: white;
                    background-color: #d35400;
                    padding: 5px;
                    border-radius: 5px;
                    box-shadow: 3px 3px 5px rgba(0,0,0,0.3);
                    ">
                    <b>{launch_site}</b>
                </div>
            '''
        )
    ).add_to(sites_fg)

# Add feature group and fit bounds
sites_fg.add_to(site_map_p)
if bounds:
    site_map.fit_bounds(bounds, padding=(24, 24))

# Layer control for toggling
folium.LayerControl(collapsed=False).add_to(site_map_p)

site_map_p


### Findings: Launch Site Locations Relative to Equator and Coastline

**Are all launch sites in proximity to the Equator line?**  
Not exactly. The mapped sites are located in the subtropics to mid-latitudes:  
- **Cape Canaveral / Kennedy Space Center (Florida)** ~28.5° N (~3,170 km from Equator)  
- **Boca Chica (Texas)** ~26.0° N (~2,900 km from Equator)  
- **Vandenberg Space Force Base (California)** ~34.6° N (~3,850 km from Equator)  

While closer proximity to the Equator is beneficial for equatorial or low-inclination orbits (due to Earth’s rotational velocity boost), U.S. launch sites are constrained by geography and mission requirements. Vandenberg, for instance, is optimal for polar and Sun-synchronous orbits despite being farther from the Equator.

**Are all launch sites in very close proximity to the coast?**  
Yes. All mapped sites are coastal or within a few kilometers of the ocean. Coastal placement:  
- Ensures safe downrange areas over open water.  
- Facilitates recovery of rocket stages and fairings.  
- Provides unobstructed ascent corridors over the ocean for various orbital inclinations.

**Key Takeaways**  
- **Equator distance**: Influences delta-v efficiency for certain mission profiles.  
- **Coastal location**: Primarily a safety, recovery, and trajectory consideration.  
- Mapping these elements highlights how geography directly impacts launch strategy.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = 28.56230197
launch_site_lon = -80.57735648
coastline_lat = 28.56426
coastline_lon = -80.56808

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Distance to coast: {distance_coastline:.2f} km")

Distance to coast: 0.93 km


In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )


# Create and add marker on the coastline point with distance label
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(120, 18),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    ),
    tooltip=f"Closest coastline point\nLat: {coastline_lat}, Lon: {coastline_lon}"
)
site_map.add_child(distance_marker)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

# Coordinates: Launch site and closest coastline
launch_site_coords = [launch_site_lat, launch_site_lon]   # CCAFS LC-40
coastline_coords   = [coastline_lat,    coastline_lon]      # Closest coastline point

# Create the polyline
lines = folium.PolyLine(
    locations=[launch_site_coords, coastline_coords],
    weight=2,
    color='blue',
    opacity=0.8
)

# Add to map
site_map.add_child(lines)



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [18]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [21]:

import folium
from folium.features import DivIcon

# ---- New, clean map for proximity analysis ----
site_map_proximity = folium.Map(location=nasa_coordinate, zoom_start=5)

# ---- Re-draw launch sites with your styled labels (no clusters here) ----
for _, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    lat = float(row['Lat'])
    lon = float(row['Long'])

    folium.Circle(
        [lat, lon],
        radius=1000,
        color='#d35400',
        fill=True,
        fill_opacity=0.15
    ).add_to(site_map_proximity)

    folium.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'''
                <div style="
                    font-size:16px;
                    color:white;
                    background-color:#d35400;
                    padding:5px;
                    border-radius:5px;
                    box-shadow:3px 3px 5px rgba(0,0,0,0.3);
                ">
                    <b>{launch_site}</b>
                </div>
            '''
        )
    ).add_to(site_map_proximity)

# ---- Proximities for CCAFS LC-40 ----
# Make sure calculate_distance(lat1, lon1, lat2, lon2) is defined from earlier cell

# Launch pad coords
pad_name = "CCAFS LC-40"
pad_lat, pad_lon = 28.562302, -80.577356

# Your provided proximity points
proximities = {
    "City: Cape Canaveral, FL (Port Canaveral)": (28.4058, -80.60648),   
    "Railway: NASA Railroad Spur (KSC Industrial)": (28.64917, -80.80667),
    "Highway: Florida State Road 401": (28.40938, -80.63212),    
}

colors = {
    "City":   "#0b7285",
    "Railway":"#2a9d8f",
    "Highway":"#e76f51",
}

# Group layer for tidy toggling (optional)
prox_fg = folium.FeatureGroup(name=f"{pad_name} — Proximities").add_to(site_map_proximity)

# Helper to add a labeled marker + line
def add_proximity(name, lat, lon, color):
    dist_km = calculate_distance(pad_lat, pad_lon, lat, lon)
    # label marker at the proximity point
    folium.Marker(
        [lat, lon],
        tooltip=f"{name} • {dist_km:.2f} km",
        icon=DivIcon(
            icon_size=(280, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px;color:{color};"><b>{name}: {dist_km:.2f} km</b></div>'
        )
    ).add_to(prox_fg)
    # polyline from pad to the point
    folium.PolyLine([[pad_lat, pad_lon], [lat, lon]], weight=2, color=color, opacity=0.85).add_to(prox_fg)
    return dist_km

# Add all proximities
dists = {}
for label, (lat, lon) in proximities.items():
    key = next(k for k in colors if label.startswith(k))
    dists[label] = add_proximity(label, lat, lon, colors[key])

# Small marker at the pad for clarity
folium.CircleMarker([pad_lat, pad_lon], radius=5, color="#264653", fill=True, fill_opacity=1,
                    tooltip=pad_name).add_to(prox_fg)

# Fit bounds around pad + points
bounds = [[pad_lat, pad_lon]] + [list(v) for v in proximities.values()]
site_map_proximity.fit_bounds(bounds, padding=(24, 24))

# Optional: show distances in notebook output
for k, v in dists.items():
    print(f"{k}: {v:.2f} km")

# Toggler
folium.LayerControl(collapsed=False).add_to(site_map_proximity)

site_map_proximity


City: Cape Canaveral, FL (Port Canaveral): 17.64 km
Railway: NASA Railroad Spur (KSC Industrial): 24.39 km
Highway: Florida State Road 401: 17.83 km


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


* **Yes**, all the listed launch sites (CCAFS LC-40, CCAFS SLC-40, KSC LC-39A, and VAFB SLC-4E) are in **very close proximity to the coast**.
* This is a deliberate and strategic choice to maximize safety, operational efficiency, and orbital flexibility.

---

## **Detailed Explanation**

### **1. Safety Considerations**

* **Flight Path Over Water:** Launching over the ocean reduces the risk to populated areas in the event of a failure during ascent.
* **Debris Management:** Rocket stages and fairings fall into the ocean rather than over land.
* **Emergency Abort Options:** Water provides a safer landing zone in case of an abort.

### **2. Orbital Mechanics**

* **Eastward Launch Advantage:** Coastal sites like those in Florida can launch eastward over the Atlantic, using Earth's rotation for a velocity boost, reducing fuel needs.
* **Trajectory Flexibility:** Coastal locations allow multiple orbital inclinations without overflying populated areas.

### **3. Logistics**

* **Marine Transport of Large Components:** Rockets and heavy launch equipment can be delivered by ship directly to the launch facility.
* **Integration Facilities Near Port:** Cape Canaveral and Kennedy Space Center are directly linked to Port Canaveral for this reason.

### **4. Examples**

* **CCAFS LC-40 & SLC-40** – Just a few kilometers from the Atlantic shoreline.
* **KSC LC-39A** – On Merritt Island, east side directly facing the ocean.
* **VAFB SLC-4E** – On the California coast, allowing southward launches over the Pacific for polar orbits.

---

## **Additional Context from Proximity Analysis**

Using the `proximities` data:

* **Close to Railways** – NASA Railroad Spur is within operational range for transporting heavy equipment.
* **Close to Highways** – Florida State Road 401 connects directly to launch sites for road transport.
* **Far from City Centers** – Launch sites maintain distance from populated areas for safety.
* **Right on the Coastline** – All sites are within a few hundred meters to a few kilometers of the ocean.



# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
